In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image, display
import nltk
from nltk.classify import apply_features
from nltk import word_tokenize
from collections import Counter
import random
import time
import re
from datetime import datetime
import pylast
import spotipy
import spotipy.util as util
fs = 16 # fontsize

## Load and format the lyrics JSON object

In [206]:
artists_rap = json.load(open('lyrics/rap_lyrics.json'))

In [207]:
artists_rock = json.load(open('lyrics/rock_lyrics.json'))

In [208]:
artists_country = json.load(open('lyrics/country_lyrics.json'))

In [210]:
artists_raw = artists_rap + artists_rock + artists_country

In [211]:
# Determine
def getArtistImage(songs):    
    urls = [None]
    for song in songs:
        if song['raw']['album'] is not None:
            try:                
                urls.append(song['raw']['album']['artist']['image_url'])                
            except:
                pass
    
    return Counter(urls).most_common()[0][0] # Most common image

# Use a generator to exclude artists with too few songs
def filterBySongCount(artists, min_count=10, max_count=10000):
    for artist in artists:        
        num_songs = len(artist['songs'])
        if (num_songs >= min_count) and (num_songs < max_count): yield artist

min_songs, max_songs = 10, 2500
artists = list(filterBySongCount(artists_raw, min_songs, max_songs))

# Artist images
for artist in artists:
    artist['image_url'] = getArtistImage(artist['songs'])
    
    # Kludge
    if artist['artist'] == 'Black Thought':
        artist['image_url'] = 'https://images.genius.com/23de511f4ac50d900128ea363b7d81b3.450x320x1.jpg'
    if artist['artist'] == 'André 3000':
        artist['image_url'] = 'http://images.genius.com/64b15c9489c65f5bf8f6577334347404.434x434x1.jpg'            
        
# Gender
labels_gender = np.array([a['gender'] for a in artists])
mask_female = np.array([g=='female' for g in labels_gender])
mask_male   = np.array([g=='male'   for g in labels_gender])
print("You have lyrics from {} artists, with a minimum of {} songs per artist.\nFemale: {}, male: {}"
      .format(len(artists), min_songs, (labels_gender=='female').sum(), (labels_gender=='male').sum()))

# Add all song lyrics to a single list
all_songs, genres = [], []
for artist, gender in zip(artists, labels_gender):
    for song in artist['songs']:        
        song['gender'] = gender
        if len(song['lyrics'].split(' ')) > 10:
            all_songs.append(song)
            if song['genre'] not in genres:
                genres.append(song['genre'])
              
song_count = len(all_songs)
print("Database contains {} songs.".format(song_count))

# Store all lyrics in a single string
all_lyrics = " ".join([song['lyrics'] for song in all_songs])

You have lyrics from 303 artists, with a minimum of 10 songs per artist.
Female: 79, male: 222
Database contains 56907 songs.


## Load the last.fm API credentials

In [8]:
with open('secrets_lastfm.txt','r') as f:
    lines = [line.split(': ') for line in f.readlines()]
    API_KEY = lines[1][1].strip()
    API_SECRET = lines[2][1].strip()        
    username = lines[3][1].strip()    
    password = lines[4][1].strip()

password_hash = pylast.md5(password)
lastfm = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET,
                              username=username, password_hash=password_hash)

## Load the Spotify API credentials

In [101]:
with open('secrets_spotify.txt','r') as f:
    lines = [line.split(': ') for line in f.readlines()]
    spotify_client_id = lines[0][1].strip()
    spotify_client_secret = lines[1][1].strip()    
    spotify_username = lines[2][1].strip()
    
scope = 'user-library-read'
spotify_token = util.prompt_for_user_token(spotify_username, scope, client_id=spotify_client_id,
                           client_secret=spotify_client_secret,
                           redirect_uri='http://localhost/')

# Create the spotipy object
sp = spotipy.Spotify(spotify_token)

In [287]:
import spotipy.oauth2 as oauth

In [288]:
spo = oauth.SpotifyOAuth(client_id=spotify_client_id,
                   client_secret=spotify_client_secret,
                   redirect_uri='http://localhost/',
                   scope=scope,
                   cache_path=".cache-{}".format(username))

In [293]:
def get_spotify_token():
    token_info = spo.get_cached_token()
    if token_info:
        access_token = token_info['access_token']
        return access_token
    else:
        auth = spo.get_authorize_url()
        print(auth)
        auth_url = input('Click the link above and copy and paste the url here: ')
        _re_auth = re.findall(_auth_finder, auth_url)
        access_token = self.spo.get_access_token(_re_auth[0])
        return access_token    

In [299]:
def refresh_spotify_token():
    cached_token = spo.get_cached_token()
    refreshed_token = cached_token['refresh_token']
    new_token = spo.refresh_access_token(refreshed_token)        
    return new_token

## Get duration and date information for all songs

In [381]:
def song_is_lyrics(song_title):
    """Returns False if result from Genius is not actually song lyrics"""
    regex = re.compile(
        r"(booklet)|(tracklist)|(track list)|(album art(work)?)|(liner notes)|(booklet)|(credits)|(remix)|(interview)|(skit)", re.IGNORECASE)
    return not regex.search(song_title)

In [354]:
spotify_artists[1].keys()

dict_keys(['artist', 'songs'])

In [358]:
combo = spotify_artists[1].copy()

In [368]:
len(spotify_artists[0]['songs'])

172

In [399]:
i

7

In [402]:
i

13

In [403]:
j

317

In [404]:
artist_name

'Kanye West'

In [406]:
j

26

In [409]:
(artist_name=='Kanye West' and j>317)

False

In [ ]:
skip_num = 13
# spotify_artists = []
start = time.time()
time_timeout = start
for i,artist in enumerate(artists):
    success_count, fail_count = 0,0
    artist_name = artist['artist']
    if artist_name not in [artist['artist'] for artist in artists[:skip_num]]:
        spotify_artists.append({'artist': artist_name, 'songs': []})
    print("{line}\nArtist: {name}".format(line="-"*60, name=artist_name))
    for j,song in enumerate(artist['songs']):
        if (artist_name not in [artist['artist'] for artist in artists[:skip_num]]):
            duration,year = None, None
            song_title = song['title']
            per = 100*success_count/(success_count+fail_count) if (success_count+fail_count)>0 else 0
            if j%10 == 0:
                print("Song: {title} || ({j}/{tot}), (s={s},f={f},p={p:.2f})".format(title=song_title, j=j, tot=len(artist['songs']),
                     s=success_count, f=fail_count, p=per))

            if song_is_lyrics(song_title): # reject non-songs            
                album = song['album'] if song['album'] is not None else ""            
                try: # Try Spotify first
                    q = "{song_title} {artist_name} {album}".format(song_title=song_title, artist_name=artist_name, album=album)
                    search_result = sp.search(q, limit=1, type='track')
                    if search_result['tracks']['total'] != 0:
                        try: 
                            track = search_result['tracks']['items'][0]                
                            duration = track['duration_ms']/1e3 # convert to seconds                
                            if 'release_date' in track['album']:
                                year = track['album']['release_date']
                            spotify_artists[i]['songs'].append({'lyrics': song['lyrics'],
                                                                'title': song_title,
                                                                'album': album,
                                                                'spotify': track})
                            artists[i]['songs'][j]['spotify'] = track
                            
                        except Exception as e:
                            print("ERROR (sp): {}".format(e))
                    else: # Search for song on last.fm if song not found on Spotify
                        try:
                            track = lastfm.get_track(artist_name, song['title'])
                            duration = track.get_duration()/1e3 # convert to seconds
                            duration = None if duration==0 else duration
                        except Exception as e:
                            if e.details != "Track not found":                        
                                print("ERROR (fm): {}".format(e))
                except spotipy.client.SpotifyException as e:
                    print("ERROR (SP): {}".format(e))
                    print("Time since last timeout: {:.2f} minutes".format((time.time()-time_timeout)/60))
                    time_timeout = time.time()
                    sp = spotipy.Spotify(auth=refresh_spotify_token()['access_token'])

                # Assign the duration and year values to the artist's song
                artists[i]['songs'][j]['duration'] = duration
                if year is not None:
                    artists[i]['songs'][j]['year'] = year

                if duration is None:
                    fail_count += 1
                else:
                    success_count += 1
                    print("{title}: ({dur}, {year})".format(title=song_title,
                                                          dur=artists[i]['songs'][j]['duration'],
                                                          year=artists[i]['songs'][j]['year']))

                # Wait before calling API again
                time.sleep(5)
    print("Total songs added for {}: {}".format(artist_name, success_count))
    
    # Save the spotify artists object
    if i%5 == 0:
        filename = 'spotify_data_v{}'.format(i)
        with open(filename + '.json', 'w') as outfile:
            json.dump(spotify_artists, outfile)
            
end = time.time()
print("Time elapsed: {} hours".format((end-start)/60/60))

------------------------------------------------------------
Artist: 2Pac
Total songs added for 2Pac: 0
------------------------------------------------------------
Artist: Eminem
Total songs added for Eminem: 0
------------------------------------------------------------
Artist: The Notorious B.I.G.
Total songs added for The Notorious B.I.G.: 0
------------------------------------------------------------
Artist: Nas
Total songs added for Nas: 0
------------------------------------------------------------
Artist: Ice Cube
Total songs added for Ice Cube: 0
------------------------------------------------------------
Artist: JAY-Z
Total songs added for JAY-Z: 0
------------------------------------------------------------
Artist: Snoop Dogg
Total songs added for Snoop Dogg: 0
------------------------------------------------------------
Artist: Dr. Dre
Total songs added for Dr. Dre: 0
------------------------------------------------------------
Artist: Kendrick Lamar
Total songs added for 

Paranoid: (277.906, 2008-01-01)
Pinocchio Story: (361.6, 2008-01-01)
Song: "Pop Style" Rant - October 20th, 2016 || (320/484), (s=132,f=138,p=48.89)
Porno (Interlude): (75.0, 2007-05-27)
Power: (292.093, 2010-01-01)
Pt. 2: (130.293, 2016-06-10)
Real Friends: (251.573, 2016-06-10)
Song: Rich Nigga Drunk* || (330/484), (s=136,f=142,p=48.92)
RoboCop: (274.36, 2008-01-01)
Roses: (245.866, 2005-09-30)
Runaway: (547.733, 2010-01-01)
Song: SAIC Commencement Lecture || (340/484), (s=139,f=149,p=48.26)
Saint Pablo: (372.113, 2016-06-10)
Say You Will: (377.6, 2008-01-01)
School Spirit: (182.386, 2004-02-13)
Song: See You in My Nightmares || (350/484), (s=142,f=154,p=47.97)
See You in My Nightmares: (258.12, 2008-01-01)
Self Conscious: (199.0, None)
Self Conscious (A.K.A. Yeah): (215.0, None)
Selfish: (227.666, 2004-03-16)
Send It Up: (178.04, 2013-01-01)
Siiiiiiiiilver Surffffeeeeer Intermission: (56.56, 2016-06-10)
Song: Skit #4 || (360/484), (s=148,f=155,p=48.84)
Slow Jamz: (316.0, 2003-12-02)

Let's Get It On: (265.0, 2002-01-01)
Life Be My Song: (242.866, 2006-08-01)
Lil Wayne Interlude: (24.006, 2012-09-11)
Lookin Without Seein (Intro): (50.132, 2012-09-11)
Look Thru My Eyes: (230.426, 1998-01-01)
Lord Give Me a Sign: (208.506, 2006-08-01)
Love That Bitch: (200.652, 2012-09-11)
Make a Move: (213.626, 1999-01-01)
Song: Mary J. Blige Diss || (150/287), (s=115,f=26,p=81.56)
Mary J. Blige Diss: (122.0, 1998-01-01)
Moe Wings: (228.445, 2016-03-11)
More 2 a Song: (222.226, 1999-01-01)
Murdergram: (294.0, 2002-01-01)
My Life: (189.333, 2003-01-01)
My Niggas: (87.333, 1998)
Never Gonna Lie: (221.0, None)
Song: Next Out the Kennel || (160/287), (s=122,f=28,p=81.33)
Niggaz Can't Touch Me Kid: (267.0, 1998-01-01)
No Love: (200.866, 2012-09-11)
No Love 4 Me: (240.0, 1998-12-22)
No Love For Me: (240.266, 1998)
No Sunshine: (317.0, 2001-01-01)
No Sunshine (Album Version): (318.0, None)
Song: Number 11 || (170/287), (s=128,f=31,p=80.50)
Number 11: (265.466, 2001-01-01)
On and On: (215.25

Song: How Much We Grew || (120/329), (s=77,f=32,p=70.64)
How Much We Grew: (294.733, 2000-06-09)
How We Do it Over Here: (216.026, 2006-01-01)
How We Roll: (249.253, 2011-01-01)
Hustler's Anthem 09: (269.333, 2009-05-19)
If You Don't Know Now You Know: (298.573, 2009-05-19)
I Got Bass: (243.0, None)
I Know What You Want: (324.306, 2002-01-01)
I'll Do it All: (302.026, 2006-01-01)
Song: I'll hurt u - remix || (130/329), (s=85,f=34,p=71.43)
I'll Hurt You: (149.0, 2006-07-13)
Ill Vibe: (209.76, 1996)
I Love My B**** - Album Version (Edited): (227.666, 2006-01-01)
I Love My Bitch: (227.666, 2006-01-01)
I'm a Go and Get My...: (294.026, 2009-05-19)
I'm Talking to You: (213.0, 2012-08-21)
In The Ghetto: (233.44, 2006-01-01)
Song: In The Ghetto (Remix) || (140/329), (s=92,f=36,p=71.88)
In the Streets: (262.0, 2015-12-25)
Intro: (284.773, 1997)
Intro (It Ain't Safe No More): (106.6, 2002-01-01)
Intro: The Current State Of Anarchy: (147.373, 2000-06-09)
Intro - there is only one year left: (157

Song: Chris Tucker || (50/260), (s=26,f=15,p=63.41)
Chris Tucker: (210.0, 2013-04-30)
Cole Summer: (292.0, 2013-04-30)
Cole World: (234.773, 2011-09-23)
College Boy: (305.0, 2007-05-04)
Cost Me A Lot: (197.0, 2010-11-12)
Crooked Smile: (278.573, 2013-06-14)
Song: Crooked Smile (Original) || (60/260), (s=32,f=18,p=64.00)
Crooked Smile (Original): (268.0, 2014-01-28)
Crunch Time: (214.0, 2013-02-12)
Daddy's Little Girl: (183.0, 2011-09-27)
Dead Presidents: (189.0, 2007-05-04)
Dead Presidents II: (214.0, 2009-06-15)
Deja Vu: (264.746, 2016-12-09)
Disgusting: (250.0, None)
Dollar and A Dream: (261.0, 2007-05-04)
Song: Dollar And A Dream III || (70/260), (s=40,f=20,p=66.67)
Dollar And A Dream III: (283.066, 2011-09-23)
Dreams: (264.0, 2009-06-15)
Enchanted: (251.0, 2010-11-12)
Everybody dies: (161.826, 2016-12-06)
False Prophets: (237.426, 2016-12-06)
Farewell: (211.0, 2010-11-12)
Fire Squad: (288.2, 2014-12-09)
Foldin Clothes: (316.92, 2016-12-09)
Song: Folgers Crystals || (80/260), (s=48,

...

You Gon Go?: (272.053, 2004)
Total songs added for KRS-One: 204
------------------------------------------------------------
Artist: Ludacris
Song: 1st & 10 || (0/231), (s=0,f=0,p=0.00)
1st & 10: (223.0, 2000-05-16)
Act a Fool: (270.733, 2003-01-01)
All the Way Turned Up (Ludamix): (349.0, None)
Area Codes: (303.026, 2001-01-01)
Badaboom: (173.0, None)
Beast Mode: (216.186, 2015-03-31)
Block Lockdown: (468.532, 2001-01-01)
Song: Blow it Out || (10/231), (s=7,f=1,p=87.50)
Blow it Out: (245.066, 2003-01-01)
Blueberry Yum Yum: (235.093, 2004-01-01)
B.O.T.S. Radio: (296.373, 2010-01-01)
Burning Bridges: (245.333, 2015-03-31)
Call Up the Homies: (243.613, 2008-01-01)
Call Ya Bluff: (187.933, 2015-03-31)
Can't Live With You: (259.64, 2010-01-01)
Catch Up: (255.0, 2000-05-16)
Song: Charge It to the Rap Game || (20/231), (s=15,f=2,p=88.24)
Charge It to the Rap Game: (230.226, 2015-03-31)
Child of the Night: (301.813, 2004-01-01)
Cold Outside: (363.32, 2001-01-01)
Come and See Me: (265.68, 2015

Amazing Amy: (300.0, 2015-01-24)
Amen: (297.0, 2006-12-02)
American Star: (216.893, 2009-01-01)
Song: A Message To The DJs || (40/1100), (s=16,f=23,p=41.03)
A Message To The DJs: (62.24, 2008-04-01)
A Milli: (221.84, 2008-01-01)
Anything: (155.0, 2005-11-20)
Army Gunz: (244.6, 2006-01-01)
Ask Dem Hoes: (255.0, 2007-10-20)
Song: Awkward || (50/1100), (s=21,f=26,p=44.68)
Awkward: (259.0, 2013-02-16)
Ay Man: (306.0, 2009-01-01)
Back To You: (329.48, 2013-01-01)
Ballin: (78.0, 2006-12-02)
Bang Bang: (239.0, 2008-11-18)
Song: Banned From T.V. || (60/1100), (s=26,f=31,p=45.61)
Banned From T.V.: (175.0, 2009-10-31)
Bass Beat: (164.0, 2005-12-13)
Beat The Shit: (268.533, 2013-01-01)
Beef: (266.493, 2009-01-01)
Believe Me: (337.0, 2014-05-06)
Song: Believe Me(Remix) || (70/1100), (s=31,f=36,p=46.27)
Believe That: (252.226, 2002-01-01)
Best Of Me: (200.124, 2011-11-28)
Best Rapper Alive: (293.68, 2005-01-01)
Song: Big Dogg Status (Lil Wayne Verse) || (80/1100), (s=34,f=42,p=44.74)
Big Tigger Liv

I'm Feeling Myself: (264.0, None)
I'm From The South: (244.0, None)
Song: I'm Gangsta || (430/1100), (s=191,f=216,p=46.93)
I'm Gangsta: (317.0, None)
I'm Good: (147.0, 2009-10-31)
I Miss My Dawgs: (275.093, 2004-01-01)
I'm Not Human: (333.04, 2010-01-01)
I'm On One: (296.146, 2011-01-01)
Song: I'm Raw || (440/1100), (s=196,f=221,p=47.00)
I'm Raw: (159.0, None)
I'm Single: (333.0, 2010-09-27)
I'm Single (Promo Only clean edit): (333.0, None)
I'm So Over You: (178.373, 2009-01-01)
I'm That Nigga: (224.0, 2015-07-04)
I'm the Best Rapper Alive: (163.266, 2014-08-28)
Song: Inside || (450/1100), (s=202,f=224,p=47.42)
Inside: (90.573, 2004-01-01)
Interlude: (121.226, 2011-01-01)
In The Booth: (291.0, 2007-06-16)
In The City: (231.173, 2008-01-01)
In The Morning: (237.807, 2012-05-07)
Intro (Tha Block Is Hot): (107.16, 1999-01-01)
Intro (Watch Them People): (33.973, 2009-01-01)
Song: Invasion || (460/1100), (s=209,f=227,p=47.94)
In Your Face: (275.626, 2010-01-01)
Itchin: (213.0, None)
It's Go

Right Above It (clean): (366.0, None)
Rob Nice Live On The Radio: (73.282, 2002-01-01)
Rock N Roll: (252.386, 2011-03-29)
Rock Star: (300.813, 2007-09-18)
Roger That: (210.0, None)
Song: Rollers & Riders || (790/1100), (s=371,f=370,p=50.07)
Rollin' (Freestyle): (178.0, None)
Romance: (260.0, 2013-03-22)
Roman Reloaded: (218.746, 2012-01-01)
Rude Boy: (209.0, 2007-04-13)
Runnin: (271.373, 2009-01-01)
Runnin circles: (263.026, 2015-06-09)
Song: Run This Town || (800/1100), (s=377,f=373,p=50.27)
Salute: (314.813, 2010-01-01)
Screwed Up: (248.999, 2010-11-30)
Song: Secretary || (810/1100), (s=379,f=381,p=49.87)
Secretary: (149.0, 2006-12-02)
Self Destruction: (241.0, None)
Selsun Blue: (216.0, 2015-01-15)
Sex Faces: (253.306, 2013-08-15)
Sex In The Lounge: (207.786, 2012-04-03)
Shades: (304.266, 2012-01-01)
Song: She's a Ryder || (820/1100), (s=385,f=385,p=50.00)
She's a Ryder: (298.066, 2008-04-01)
She Tried: (160.386, 2013-04-16)
She Will: (305.84, 2011-01-01)
Shine: (304.173, 2009-01-01

Back Where I Belong: (263.0, 2000-09-12)
Bartender Please: (276.0, 2013-04-25)
Bath Salt: (254.0, None)
Best Dress: (237.866, 2006-04-11)
Between the Sheetz: (230.0, None)
Song: Big Mama (Unconditional Love) || (30/224), (s=25,f=3,p=89.29)
Big Mama (Unconditional Love): (334.2, 2002-10-08)
Big Ole Butt: (277.0, 1989-06-09)
Blink Blink: (240.0, None)
Born to Love You: (222.226, 2002-10-08)
Buckin' Em Down: (241.666, 1993-06-01)
Bump This: (265.0, None)
Candy: (272.323, 1997-01-01)
Can't Explain It: (252.626, 2004-08-31)
Can't Think: (289.0, 2000-09-12)
Change Your Ways: (200.0, 1989-06-09)
Song: Cheesy Rat Blues || (40/224), (s=35,f=3,p=92.11)
Cheesy Rat Blues: (305.586, 1990-08-27)
Clap Your Hands: (307.0, 1989-06-09)
Clockin' G's: (248.973, 2002-10-08)
Closer: (228.0, None)
Come and Party With Me: (276.786, 2008-01-01)
Control Myself: (233.973, 2006-04-11)
Crazy Girl: (237.0, None)
Crime Stories: (190.0, None)
Crossroads: (329.4, 1993-06-01)
Cry: (255.026, 2008-01-01)
Song: Dangerous 

In [408]:
j

0

In [396]:
j

77

In [397]:
filename = 'spotify_data_b3'
with open(filename + '.json', 'w') as outfile:
    json.dump(spotify_artists, outfile)
